In [ ]:

import json
from IPython.display import Markdown,display
from openai import OpenAI

In [ ]:
openAI=OpenAI()
Model='llama3.2'

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
link_user_prompt=f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""

In [ ]:
links="https://www.udemy.com/course/llm-engineering-master-ai-and-large-language-models"

In [ ]:
def combine_link(links):
    link_user_prompt+=links
    return  link_user_prompt

In [ ]:
def select_relevant_links(url):
 
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": combine_link(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
def get_brouchere_user_prompt(company_name,url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt+=select_relevant_links(url)
    user_prompt=user_prompt[:5000]
    return user_prompt


In [ ]:
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": combine_link(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    